# Creating a Segmentation App with MONAI Deploy App SDK

This tutorial shows how to create an organ segmentation application for a PyTorch model that has been trained with MONAI and packaged into a MONAI bundle. This is derived form the segmentation app tutorial notebook.

### Setup environment


In [1]:
%pip install --upgrade monai-deploy-app-sdk
%pip install monai # for MONAI transforms

### Download/Extract ai_spleen_seg_data from Google Drive

In [1]:
# Download ai_spleen_seg_data test data zip file
!pip install gdown 
!gdown "https://drive.google.com/uc?id=1GC_N8YQk_mOWN02oOzAU_2YDmNRWk--n"

# After downloading ai_spleen_seg_data zip file from the web browser or using gdown,
!unzip -qo "ai_spleen_seg_data_updated_1203.zip"

Downloading...
From: https://drive.google.com/uc?id=1GC_N8YQk_mOWN02oOzAU_2YDmNRWk--n
To: /home/localek10/workspace/monai/monai-deploy-app-sdk/notebooks/bundles/ai_spleen_seg_data_updated_1203.zip
100%|████████████████████████████████████████| 104M/104M [00:03<00:00, 26.7MB/s]


## Create Bundle From Torchscript Object

In [3]:
!mkdir -p spleen_segmentation
!mkdir -p spleen_segmentation/configs
!mkdir -p spleen_segmentation/models

In [7]:
import torch
obj=torch.jit.load("model.ts")
state=obj.state_dict()
torch.save({k:v.clone().cpu() for k,v in state.items()},"spleen_segmentation/models/model.pt")

/home/localek10/miniconda3/envs/deploy/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [12]:
%%writefile spleen_segmentation/configs/metadata.json

{
    "schema": "https://github.com/Project-MONAI/MONAI-extra-test-data/releases/download/0.8.1/meta_schema_20220324.json",
    "version": "0.1.0",
    "changelog": { "0.0.1": "initialize the model package structure"},
    "monai_version": "0.8.0",
    "pytorch_version": "1.10.0",
    "numpy_version": "1.21.2",
    "optional_packages_version": {    },
    "network_def": {
        "_target_": "UNet",
        "spatial_dims": 3,
        "in_channels": 1,
        "out_channels": 2,
        "channels": [16, 32, 64, 128, 256],
        "strides": [2, 2, 2, 2],
        "num_res_units": 2,
        "norm": "batch"
    },
    "task": "Spleen Segmentation",
    "description": "A pre-trained model for segmenting the spleen",
    "authors": "MONAI team",
    "copyright": "Copyright (c) MONAI Consortium",
    "data_source": "Speen data from the Medical Segmentation Decathlon",
    "data_type": "dicom",
    "image_classes": "single channel data, intensity scaled to [0, 1]",
    "label_classes": "single channel data, 0 is background, 1 is segmentation",
    "pred_classes": "2 channel probability data",
    "intended_use": "This is an example, not to be used for diagnostic purposes",
    "network_data_format": {
        "inputs": {
            "image": {
                "type": "image",
                "format": "magnitude",
                "num_channels": 1,
                "spatial_shape": [160, 160, 160],
                "dtype": "float32",
                "value_range": [],
                "is_patch_data": true,
                "channel_def": {"0": "image"}
            }
        },
        "outputs": {
            "pred": {
                "type": "image",
                "format": "segmentation",
                "num_channels": 2,
                "spatial_shape": [160,160,160],
                "dtype": "float32",
                "value_range": [],
                "is_patch_data": true,
                "channel_def": {
                    "0": "background",
                    "1": "foreground"
                }
            }
        }
    }
}

Overwriting spleen_segmentation/configs/metadata.json


In [10]:
%%writefile spleen_segmentation/configs/inference.json

{
    "imports": [
        "$import glob",
        "$import os"
    ],
    "device": "$torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')",
    "ckpt_path": "./spleen_segmentation/models/model.pt",
    "dataset_dir": "/workspace/data",
    "datalist": "$list(sorted(glob.glob(@dataset_dir + '/*/*.nii')))",
    "network_def": {
        "_target_": "UNet",
        "spatial_dims": 3,
        "in_channels": 1,
        "out_channels": 2,
        "channels": [16, 32, 64, 128, 256],
        "strides": [2, 2, 2, 2],
        "num_res_units": 2,
        "norm": "batch"
    },
    "network": "$@network_def.to(@device)",
    "preprocessing": {
        "_target_": "Compose",
        "transforms": [
            {
                "_target_": "LoadImaged",
                "keys": "image"
            },
            {
                "_target_": "EnsureChannelFirstd",
                "keys": "image"
            },
            {
                "_target_": "Orientationd",
                "keys": "image",
                "axcodes": "RAS"
            },
            {
                "_target_": "Spacingd",
                "keys": "image",
                "pixdim": [1.0, 1.0, 1.0],
                "mode": ["bilinear"],
                "align_cornders": true
            },
            {
                "_target_": "ScaleIntensityRanged",
                "keys": "image",
                "a_min":-57, 
                "a_max":164, 
                "b_min":0.0, 
                "b_max":1.0, 
                "clip": true
            },
            {
                "_target_": "EnsureTyped",
                "keys": "image"
            }
        ]
    },
    "dataset": {
        "_target_": "Dataset",
        "data": "$[{'image': i} for i in @datalist]",
        "transform": "@preprocessing"
    },
    "dataloader": {
        "_target_": "DataLoader",
        "dataset": "@dataset",
        "batch_size": 1,
        "shuffle": false,
        "num_workers": 0
    },
    "inferer": {
        "_target_": "SlidingWindowInferer",
        "roi_size": [160, 160, 160],
        "sw_batch_size": 4,
        "device": "@device"
    },
    "postprocessing": {
        "_target_": "Compose",
        "transforms": [
            {
                "_target_": "Activationsd",
                "keys": "pred",
                "softmax": true
            },
            {
                "_target_": "AsDiscreted",
                "keys": "pred",
                "argmax": true
            },
            {
                "_target_": "Invertd",
                "keys": "pred",
                "orig_keys": "image",
                "nearest_interp": true,
                "transform": "@preprocessing"
            },
            {
                "_target_": "SaveImaged",
                "keys": "pred",
                "output_dir": "output",
                "output_postfix": "seg",
                "output_dtype": "uint8",
                "resample": false
            }
        ]
    },
    "handlers": [
        {
            "_target_": "CheckpointLoader",
            "_disabled_": "$not os.path.exists(@ckpt_path)",
            "load_path": "@ckpt_path",
            "load_dict": {"model": "@network"}
        }
    ],
    "evaluator": {
        "_target_": "SupervisedEvaluator",
        "device": "@device",
        "val_data_loader": "@dataloader",
        "network": "@network",
        "inferer": "@inferer",
        "postprocessing": "@postprocessing",
        "val_handlers": "@handlers",
        "amp": true
    }
}

Overwriting spleen_segmentation/configs/inference.json


In [13]:
!python  -m monai.bundle ckpt_export network_def \
    --filepath spleen_segmentation.ts \
    --ckpt_file spleen_segmentation/models/model.pt \
    --meta_file spleen_segmentation/configs/metadata.json \
    --config_file spleen_segmentation/configs/inference.json

2022-05-12 15:48:39,901 - INFO - --- input summary of monai.bundle.scripts.ckpt_export ---
2022-05-12 15:48:39,901 - INFO - > net_id: 'network_def'
2022-05-12 15:48:39,902 - INFO - > filepath: 'spleen_segmentation.ts'
2022-05-12 15:48:39,902 - INFO - > meta_file: 'spleen_segmentation/configs/metadata.json'
2022-05-12 15:48:39,902 - INFO - > config_file: 'spleen_segmentation/configs/inference.json'
2022-05-12 15:48:39,902 - INFO - > ckpt_file: 'spleen_segmentation/models/model.pt'
2022-05-12 15:48:39,902 - INFO - ---


'dst' model updated: 148 of 148 variables.
2022-05-12 15:48:40,441 - INFO - exported to TorchScript file: spleen_segmentation.ts.


In [3]:
from monai.deploy.core import Application, resource, IOType
from monai.deploy.operators import BundleOperator, create_bundle_operator
from monai.deploy.operators.dicom_data_loader_operator import DICOMDataLoaderOperator
from monai.deploy.operators.dicom_seg_writer_operator import DICOMSegmentationWriterOperator
from monai.deploy.operators.dicom_series_selector_operator import DICOMSeriesSelectorOperator
from monai.deploy.operators.dicom_series_to_volume_operator import DICOMSeriesToVolumeOperator

@resource(cpu=1, gpu=1, memory="7Gi")
class AISpleenSegApp(Application):
    def compose(self):

        study_loader_op = DICOMDataLoaderOperator()
        series_selector_op = DICOMSeriesSelectorOperator()
        series_to_vol_op = DICOMSeriesToVolumeOperator()
        # Creates DICOM Seg writer with segment label name in a string list
        dicom_seg_writer = DICOMSegmentationWriterOperator(seg_labels=["Spleen"])

        # Creates the model specific segmentation operator
        print(self._context.model_path)
        spleen_seg_op = create_bundle_operator(self._context.model_path, "inference",out_type=IOType.DISK)

        # Creates the DAG by linking the operators
        self.add_flow(study_loader_op, series_selector_op, {"dicom_study_list": "dicom_study_list"})
        self.add_flow(series_selector_op, series_to_vol_op, {"study_selected_series_list": "study_selected_series_list"})
        self.add_flow(series_to_vol_op, spleen_seg_op, {"image": "image"})

        # self.add_flow(series_selector_op, dicom_seg_writer, {"study_selected_series_list": "study_selected_series_list"})
        # self.add_flow(spleen_seg_op, dicom_seg_writer, {"seg_image": "seg_image"})

In [4]:
app = AISpleenSegApp()

app.run(input="dcm", output="output", model="spleen_segmentation.ts")

models


IOMappingError: The downstream operator(DummyOperator) has no input port with label 'image'. It should be one of ().